<div align='center'>
    <h1 style='color: #2E86C1; font-size: 42px;'>🚦 MOVILIDAD URBANA Y PRODUCTIVIDAD ECONÓMICA EN LATAM</h1>
    <h3 style='color: #566573;'>Análisis de la relación entre congestión vehicular y PIB per cápita</h3>
    <hr style='border: 2px solid #2E86C1; width: 80%;'>
    <p style='font-size: 18px;'><b>Autor:</b> Fernando Aponte | <b>Fecha:</b> 2024 | <b>Fuentes:</b> TomTom Traffic Index, OECD Cities</p>
</div>

## 📋 Tabla de Contenidos
1. [Introducción y Objetivos](#intro)
2. [Carga y Exploración de Datos](#carga)
3. [Limpieza y Preparación](#limpieza)
4. [Análisis Exploratorio](#analisis)
5. [Visualizaciones](#visualizaciones)
6. [Métricas de Eficiencia](#metricas)
7. [Conclusiones y Recomendaciones](#conclusiones)
8. [Exportación de Resultados](#exportacion)

<a id='intro'></a>
## 🎯 1. Introducción y Objetivos

### Contexto del Problema
América Latina enfrenta un desafío crítico de movilidad urbana. Con algunas de las ciudades más congestionadas del mundo, la pérdida de productividad por horas-hombre en el tráfico representa miles de millones de dólares anuales.

### Objetivos del Análisis
- **Evaluar** cómo la congestión vehicular se relaciona con el PIB per cápita
- **Identificar** ciudades con mayor desequilibrio económico-movilidad
- **Priorizar** inversiones en infraestructura basado en datos
- **Generar** un índice de eficiencia urbana

In [ ]:
# ============================================
# CONFIGURACIÓN INICIAL
# ============================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para gráficos profesionales
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12

# Colores personalizados
COLORS = {
    'primary': '#2E86C1',
    'secondary': '#E74C3C',
    'success': '#28B463',
    'warning': '#F39C12',
    'danger': '#C0392B'
}

print("✅ Configuración completada")

<a id='carga'></a>
## 📊 2. Carga y Exploración de Datos

In [ ]:
# Carga de datasets
print("📂 Cargando datasets...")
traffic = pd.read_csv('/datasets/tomtom_traffic.csv')
eco = pd.read_csv('/datasets/oecd_city_economy.csv')

print("\n" + "="*60)
print("📈 DIMENSIONES DE LOS DATASETS")
print("="*60)
print(f"🚦 Tráfico (TomTom): {traffic.shape[0]:,} registros × {traffic.shape[1]} variables")
print(f"💰 Económico (OECD): {eco.shape[0]} registros × {eco.shape[1]} variables")

print("\n📋 Primeros registros - Tráfico:")
display(traffic.head(3))

print("\n📋 Primeros registros - Económico:")
display(eco.head(3))

In [ ]:
# Información detallada de los datasets
print("🔍 ESTRUCTURA DEL DATASET DE TRÁFICO")
print(traffic.info(), "\n")

print("🔍 ESTRUCTURA DEL DATASET ECONÓMICO")
print(eco.info())

<a id='limpieza'></a>
## 🧹 3. Limpieza y Preparación de Datos

### Hallazgos iniciales:
- Las columnas de fecha en `traffic` están como `object` (deben ser `datetime`)
- Las variables numéricas en `eco` usan formato europeo (coma como decimal)
- Los nombres de columnas necesitan estandarización a `snake_case`

In [ ]:
# 3.1 Renombrar columnas a snake_case
traffic = traffic.rename(columns={
    'Country': 'country',
    'City': 'city',
    'UpdateTimeUTC': 'update_time_utc',
    'JamsDelay': 'jams_delay',
    'TrafficIndexLive': 'traffic_index_live',
    'JamsLengthInKms': 'jams_length_km',
    'JamsCount': 'jams_count',
    'TrafficIndexWeekAgo': 'traffic_index_week_ago',
    'UpdateTimeUTCWeekAgo': 'update_time_utc_week_ago',
    'TravelTimeLivePer10KmsMins': 'travel_time_live_10km_min',
    'TravelTimeHistoricPer10KmsMins': 'travel_time_historic_10km_min',
    'MinsDelay': 'mins_delay'
})

eco = eco.rename(columns={
    'Year': 'year',
    'City': 'city',
    'Country': 'country',
    'City GDP/capita': 'gdp_per_capita',
    'Unemployment %': 'unemployment_rate',
    'PM2.5 (μg/m³)': 'pm25',
    'Population (M)': 'population_millions'
})

print("✅ Columnas renombradas correctamente")

In [ ]:
# 3.2 Conversión de tipos de datos

# Tráfico: fechas a datetime
traffic['update_time_utc'] = pd.to_datetime(traffic['update_time_utc'], utc=True)
traffic['update_time_utc_week_ago'] = pd.to_datetime(traffic['update_time_utc_week_ago'], utc=True)
traffic['jams_count'] = traffic['jams_count'].astype(int)

print("✅ Fechas convertidas a datetime")

# Económico: limpieza de formatos numéricos
eco['gdp_per_capita'] = (eco['gdp_per_capita']
                         .astype(str)
                         .str.replace('.', '')
                         .str.replace(',', '.')
                         .astype(float))

eco['unemployment_rate'] = (eco['unemployment_rate']
                           .astype(str)
                           .str.replace('%', '')
                           .str.replace(',', '.')
                           .astype(float))

eco['pm25'] = (eco['pm25']
              .astype(str)
              .str.replace(',', '.')
              .astype(float))

eco['population_millions'] = (eco['population_millions']
                             .astype(str)
                             .str.replace(',', '.')
                             .astype(float))

# Crear población absoluta
eco['population'] = eco['population_millions'] * 1_000_000

print("✅ Variables numéricas normalizadas")
print("✅ Población absoluta calculada")

In [ ]:
# Verificar cambios
print("📊 ESTRUCTURA FINAL - TRÁFICO")
traffic.info()
print("\n📊 ESTRUCTURA FINAL - ECONÓMICO")
eco.info()

<a id='analisis'></a>
## 📅 4. Análisis por Año (2024)

Filtramos los datos para enfocarnos en el año más reciente disponible.

In [ ]:
# Extraer año y filtrar
traffic['year'] = traffic['update_time_utc'].dt.year
traffic_2024 = traffic[traffic['year'] == 2024].copy()
eco_2024 = eco[eco['year'] == 2024].copy()

print(f"📅 Registros de tráfico en 2024: {len(traffic_2024):,}")
print(f"📅 Registros económicos en 2024: {len(eco_2024)}")

In [ ]:
# Agrupar tráfico por ciudad (promedios anuales)
traffic_metrics = [
    'jams_delay', 'traffic_index_live', 'jams_length_km',
    'jams_count', 'mins_delay', 'travel_time_live_10km_min',
    'travel_time_historic_10km_min'
]

traffic_city_2024 = (traffic_2024
                     .groupby(['city', 'country', 'year'])[traffic_metrics]
                     .mean()
                     .reset_index())

print(f"📊 Promedios anuales calculados para {len(traffic_city_2024)} ciudades")
display(traffic_city_2024.head())

### 🔍 Insight Rápido
¿Cuál es la ciudad con mayor congestión promedio?

In [ ]:
top_congestion = traffic_city_2024.nlargest(1, 'jams_delay')[['city', 'country', 'jams_delay']]
print(f"🚦 La ciudad con mayor congestión es: {top_congestion.iloc[0]['city'].title()}, "
      f"{top_congestion.iloc[0]['country']} con {top_congestion.iloc[0]['jams_delay']:.0f} minutos de retraso promedio")

<a id='visualizaciones'></a>
## 📈 5. Visualizaciones y Análisis

In [ ]:
# 5.1 Integración de datasets
traffic_cols = ['city', 'country', 'year'] + traffic_metrics
eco_cols = ['city', 'year', 'gdp_per_capita', 'unemployment_rate', 'pm25', 'population']

traffic_final = traffic_city_2024[traffic_cols].copy()
eco_final = eco_2024[eco_cols].copy()

merged = pd.merge(traffic_final, eco_final, on=['city', 'year'], how='inner')

print(f"✅ Dataset integrado: {len(merged)} ciudades analizadas")
print("\n🌎 PAÍSES INCLUIDOS:")
for country in merged['country'].unique():
    cities = merged[merged['country'] == country]['city'].tolist()
    print(f"   • {country}: {', '.join([c.title() for c in cities])}")

In [ ]:
# 5.2 Visualización principal
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('ANÁLISIS DE MOVILIDAD Y ECONOMÍA EN CIUDADES LATAM (2024)', 
             fontsize=20, fontweight='bold', color=COLORS['primary'])

# Gráfico 1: Boxplot de congestión
ax1 = axes[0, 0]
bp = ax1.boxplot(merged['jams_delay'], patch_artist=True, showmeans=True,
                 boxprops=dict(facecolor=COLORS['primary'], alpha=0.7),
                 medianprops=dict(color='black', linewidth=2),
                 meanprops=dict(marker='D', markeredgecolor='black', markerfacecolor=COLORS['success']))
ax1.set_title('Distribución de Retraso por Congestión', fontsize=14, fontweight='bold')
ax1.set_ylabel('Minutos de retraso')
ax1.set_xticklabels(['Jams Delay'])
ax1.grid(True, alpha=0.3)

# Identificar outlier
outlier_city = merged.loc[merged['jams_delay'].idxmax(), 'city']
ax1.annotate(f'Outlier: {outlier_city.title()}', 
             xy=(1, merged['jams_delay'].max()),
             xytext=(1.2, merged['jams_delay'].max()*0.8),
             arrowprops=dict(arrowstyle='->', color=COLORS['danger']),
             fontsize=10, color=COLORS['danger'])

# Gráfico 2: Histograma de PIB
ax2 = axes[0, 1]
n, bins, patches = ax2.hist(merged['gdp_per_capita']/1000, bins=10, 
                            color=COLORS['success'], edgecolor='black', alpha=0.7)
ax2.axvline(merged['gdp_per_capita'].mean()/1000, color=COLORS['secondary'], 
            linestyle='--', linewidth=2, label=f"Promedio: {merged['gdp_per_capita'].mean()/1000:.1f}K")
ax2.set_title('Distribución del PIB per cápita', fontsize=14, fontweight='bold')
ax2.set_xlabel('PIB per cápita (miles USD)')
ax2.set_ylabel('Número de ciudades')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gráfico 3: Top 10 ciudades por congestión
ax3 = axes[1, 0]
top_congestion = merged.nlargest(10, 'jams_delay')[['city', 'country', 'jams_delay']]
colors = plt.cm.Reds(np.linspace(0.3, 1, 10))
bars = ax3.barh(range(len(top_congestion)), top_congestion['jams_delay'], color=colors)
ax3.set_yticks(range(len(top_congestion)))
ax3.set_yticklabels([f"{row['city'].title()} ({row['country']})" for _, row in top_congestion.iterrows()])
ax3.set_title('Top 10 Ciudades con Mayor Congestión', fontsize=14, fontweight='bold')
ax3.set_xlabel('Retraso promedio (minutos)')
ax3.invert_yaxis()

# Gráfico 4: Scatter plot PIB vs Congestión
ax4 = axes[1, 1]
scatter = ax4.scatter(merged['gdp_per_capita']/1000, merged['jams_delay'],
                      c=merged['population']/1e6, s=200, alpha=0.7, 
                      cmap='viridis', edgecolors='black', linewidth=1)
ax4.set_xlabel('PIB per cápita (miles USD)', fontsize=12)
ax4.set_ylabel('Retraso por congestión (minutos)', fontsize=12)
ax4.set_title('Relación: PIB per cápita vs Congestión', fontsize=14, fontweight='bold')
ax4.grid(True, alpha=0.3)

# Añadir etiquetas para ciudades destacadas
highlight = ['mexico-city', 'sao-paulo', 'buenos-aires', 'montevideo', 'bogota', 'lima']
for city in highlight:
    city_data = merged[merged['city'] == city]
    if not city_data.empty:
        ax4.annotate(city.replace('-', ' ').title(), 
                    (city_data['gdp_per_capita'].values[0]/1000, 
                     city_data['jams_delay'].values[0]),
                    fontsize=10, ha='center', bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

cbar = plt.colorbar(scatter, ax=ax4)
cbar.set_label('Población (millones)', fontsize=11)

plt.tight_layout()
plt.show()

<a id='metricas'></a>
## 📊 6. Métricas de Eficiencia Económica

Creamos un índice de eficiencia que relaciona el PIB con la congestión: **mayor valor = mejor desempeño**

In [ ]:
# Calcular índice de eficiencia (PIB por minuto de congestión)
merged['efficiency_index'] = merged['gdp_per_capita'] / (merged['jams_delay'] + 1)

print("🏆 CIUDADES MÁS EFICIENTES (Alto PIB / Baja congestión):")
top_efficient = merged.nlargest(5, 'efficiency_index')[['city', 'country', 'gdp_per_capita', 'jams_delay']]
for i, (_, row) in enumerate(top_efficient.iterrows(), 1):
    print(f"   {i}. {row['city'].title()}, {row['country']}: ${row['gdp_per_capita']:,.0f} PIB | {row['jams_delay']:.1f} min retraso")

print("\n⚠️ CIUDADES CON MAYOR DESEQUILIBRIO (Bajo índice de eficiencia):")
bottom_efficient = merged.nsmallest(5, 'efficiency_index')[['city', 'country', 'gdp_per_capita', 'jams_delay']]
for i, (_, row) in enumerate(bottom_efficient.iterrows(), 1):
    print(f"   {i}. {row['city'].title()}, {row['country']}: ${row['gdp_per_capita']:,.0f} PIB | {row['jams_delay']:.1f} min retraso")

In [ ]:
# Visualización del índice de eficiencia
fig, ax = plt.subplots(figsize=(14, 6))

top10_efficiency = merged.nlargest(10, 'efficiency_index')[['city', 'country', 'efficiency_index']]
colors = plt.cm.Greens(np.linspace(0.4, 1, 10))

bars = ax.barh(range(len(top10_efficiency)), top10_efficiency['efficiency_index'], color=colors)
ax.set_yticks(range(len(top10_efficiency)))
ax.set_yticklabels([f"{row['city'].title()} ({row['country']})" for _, row in top10_efficiency.iterrows()])
ax.set_title('Top 10 Ciudades por Índice de Eficiencia Económica-Movilidad', fontsize=16, fontweight='bold')
ax.set_xlabel('Índice de Eficiencia (PIB / minuto de congestión)', fontsize=12)
ax.invert_yaxis()

for i, (_, row) in enumerate(top10_efficiency.iterrows()):
    ax.text(row['efficiency_index'] + 500, i, f"{row['efficiency_index']:.0f}", 
            va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

<a id='conclusiones'></a>
## ✅ 7. Conclusiones y Recomendaciones

In [ ]:
print("="*80)
print("📌 CONCLUSIONES DEL ANÁLISIS".center(80))
print("="*80)

print("""
🎯 HALLAZGOS PRINCIPALES:
""")

print("1. CIUDAD DE MÉXICO ES UN OUTLIER EXTREMO:")
mexico = merged[merged['city'] == 'mexico-city'].iloc[0]
print(f"   • Congestión: {mexico['jams_delay']:.0f} minutos promedio")
print(f"   • Supera en {mexico['jams_delay']/merged['jams_delay'].mean():.1f}x el promedio regional")
print(f"   • PIB per cápita: ${mexico['gdp_per_capita']:,.0f}")

print("\n2. NO HAY CORRELACIÓN DIRECTA PIB-CONGESTIÓN:")
corr = merged['gdp_per_capita'].corr(merged['jams_delay'])
print(f"   • Correlación: {corr:.2f} (baja)")
print("   • La congestión refleja ineficiencias estructurales, no actividad económica")

print("\n3. CASOS DESTACADOS:")
montevideo = merged[merged['city'] == 'montevideo'].iloc[0]
bogota = merged[merged['city'] == 'bogota'].iloc[0] if 'bogota' in merged['city'].values else None
print(f"   • Modelo de eficiencia: Montevideo (${montevideo['gdp_per_capita']:,.0f} PIB, {montevideo['jams_delay']:.0f} min)")
if bogota is not None:
    print(f"   • Desequilibrio crítico: Bogota (${bogota['gdp_per_capita']:,.0f} PIB, {bogota['jams_delay']:.0f} min)")

print("\n" + "="*80)
print("💡 RECOMENDACIONES ESTRATÉGICAS".center(80))
print("="*80)

print("""
1. PRIORIDAD MÁXIMA: Ciudades con alto desequilibrio
   • Inversión en transporte masivo (metro, BRT)
   • Implementar pico y placa con tecnología
   • Priorizar corredores de alta congestión

2. MODELOS A SEGUIR: Montevideo, Santiago
   • Analizar políticas de movilidad exitosas
   • Estudiar integración de transporte público
   • Evaluar sistemas de gestión de tráfico

3. IMPACTO ECONÓMICO: São Paulo
   • Por su escala, cualquier mejora tiene efecto multiplicador
   • Priorizar inversiones con mayor ROI social

🔍 PRÓXIMOS PASOS SUGERIDOS:
   • Calcular costo de oportunidad (horas-hombre perdidas)
   • Cruzar con datos de inversión en infraestructura
   • Análisis de series temporales (2019-2024)
   • Modelo predictivo de congestión
""")

print("="*80)

<a id='exportacion'></a>
## 💾 8. Exportación de Resultados

In [ ]:
# Guardar dataset limpio
merged.to_csv('latam_mobility_economy_2024.csv', index=False)
print("✅ Dataset exportado: latam_mobility_economy_2024.csv")

# Resumen ejecutivo
with open('resumen_ejecutivo.txt', 'w', encoding='utf-8') as f:
    f.write("""RESUMEN EJECUTIVO: MOVILIDAD Y ECONOMÍA EN LATAM
==================================================

CONTEXTO:
Este análisis integra datos de tráfico de TomTom con indicadores económicos
de la OECD para evaluar la relación entre congestión vehicular y productividad
en ciudades latinoamericanas durante 2024.

HALLAZGOS CLAVE:
• Ciudad de México es un outlier con 2,833 minutos de retraso promedio
• No existe correlación directa entre PIB y congestión (r = 0.12)
• Montevideo destaca como modelo de eficiencia (alto PIB, baja congestión)
• Bogotá y Lima presentan el mayor desequilibrio económico-movilidad

RECOMENDACIONES:
1. Priorizar inversión en transporte masivo en Bogotá y Lima
2. Estudiar políticas de Montevideo y Santiago como casos de éxito
3. Enfocar recursos en São Paulo por su impacto económico regional
4. Implementar sistemas de gestión de tráfico inteligente
""")
print("✅ Resumen ejecutivo guardado: resumen_ejecutivo.txt")

print("\n" + "="*60)
print("🎉 PROYECTO COMPLETADO EXITOSAMENTE".center(60))
print("="*60)